In [1]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor

In [2]:
train_path = '/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv'
test_path = '/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv'

X_train_full = pd.read_csv(train_path)
X_test_full = pd.read_csv(test_path)

In [3]:
def clean_dataset(df):
    """Replace 'not available in demo dataset' with np.nan"""
    df_cleaned = df.replace('not available in demo dataset', np.nan)
    return df_cleaned

def remove_high_null_columns(df, threshold=40):
    """Drop columns with null percentage > threshold + additional drops"""
    nulls_percentage = df.isnull().sum() / len(df) * 100
    high_null_columns = nulls_percentage[nulls_percentage > threshold].index.tolist()
    additional_drops = ['totals.visits', 'locationZone']
    columns_to_drop = high_null_columns + additional_drops
    columns_to_drop = [col for col in columns_to_drop if col in df.columns]
    df_cleaned = df.drop(columns_to_drop, axis=1)
    print(f"Dropped columns: {columns_to_drop}")
    return df_cleaned

def handle_new_visits(df):
    
    if 'new_visits' in df.columns:
        df['new_visits'] = df['new_visits'].fillna(0)
    return df

def handle_traffic_source(df):
   
    if 'trafficSource' in df.columns:
        top_10_traffic = df['trafficSource'].value_counts().nlargest(10).index.tolist()
        df['trafficSource'] = df['trafficSource'].apply(lambda x: x if x in top_10_traffic else 'Google')
    return df

def extract_date_features(df):
   
    if 'date' in df.columns:
        date_series = pd.to_datetime(df['date'], format='%Y%m%d')
        df['year'] = date_series.dt.year
        df['month'] = date_series.dt.month
        df['day'] = date_series.dt.day
        df['day_of_week'] = date_series.dt.dayofweek
        df['quarter'] = date_series.dt.quarter
        df['is_weekend'] = (date_series.dt.dayofweek >= 5).astype(int)
        df = df.drop('date', axis=1)
    return df

def clean_unnecessary_columns(df):
   
    unnecessary_cols = [
        'userId', 'sessionId', 'sessionStart',
        'trafficSource.campaign', 'trafficSource'
    ]
    cols_to_drop = [col for col in unnecessary_cols if col in df.columns]
    if cols_to_drop:
        df = df.drop(cols_to_drop, axis=1)
        print(f"Dropped unnecessary columns: {cols_to_drop}")
    return df

def preprocess_data_complete(df):
   
    print("Starting preprocessing...")
    df = clean_dataset(df)
    print("Cleaned dataset")
    df = remove_high_null_columns(df)
    print("Removed high null columns")
    df = handle_new_visits(df)
    print("Handled new_visits")
    df = handle_traffic_source(df)
    print("Handled traffic source")
    df = extract_date_features(df)
    print("Extracted date features")
    df = clean_unnecessary_columns(df)
    print("Removed unnecessary columns")
    print("Preprocessing completed!")
    return df


In [4]:
def create_column_transformer():
    # Define numerical columns based on domain knowledge after preprocessing
    # Adjust these lists if needed based on actual dataframe columns
    numeric_cols = ['sessionNumber', 'pageViews', 'totalHits']
    categorical_cols = [
        'year','month','day','day_of_week','quarter',
        'browser', 'screenSize', 'geoCluster', 'geoNetwork.networkDomain',
        'os','geoNetwork.subContinent', 'trafficSource.medium', 'locationCountry',
        'socialEngagementType', 'deviceType', 'userChannel', 'geoNetwork.continent'
    ]
    
    numeric_transformer = make_pipeline(
        KNNImputer(n_neighbors=5),
        StandardScaler()
    )
    
    categorical_transformer = make_pipeline(
        SimpleImputer(strategy='most_frequent'),
        OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    )
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_cols),
            ('cat', categorical_transformer, categorical_cols)
        ],
        remainder='drop'  # Drop any other columns
    )
    
    return preprocessor

In [5]:
def create_complete_pipeline():
    def preprocess_step(X):
        return preprocess_data_complete(X.copy())
    
    pipeline = Pipeline([
        ('preprocessing', FunctionTransformer(preprocess_step, validate=False)),
        ('column_transformer', create_column_transformer())
    ])
    return pipeline


In [6]:
complete_pipeline = create_complete_pipeline()

# Fit pipeline on full training dataframe except the target
X_train_features = X_train_full.drop('purchaseValue', axis=1)
y_train = X_train_full['purchaseValue'].copy()

# Fit and transform training data
X_train_processed = complete_pipeline.fit_transform(X_train_features)

# Transform test data
X_test_processed = complete_pipeline.transform(X_test_full)

print(f"Processed training shape: {X_train_processed.shape}")
print(f"Processed test shape: {X_test_processed.shape}")

Starting preprocessing...


/tmp/ipykernel_13/742928516.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned = df.replace('not available in demo dataset', np.nan)


Cleaned dataset
Dropped columns: ['trafficSource.isTrueDirect', 'device.screenResolution', 'trafficSource.adContent', 'trafficSource.keyword', 'trafficSource.adwordsClickInfo.slot', 'device.mobileDeviceBranding', 'device.mobileInputSelector', 'device.mobileDeviceMarketingName', 'device.operatingSystemVersion', 'device.flashVersion', 'geoNetwork.region', 'geoNetwork.networkLocation', 'trafficSource.adwordsClickInfo.isVideoAd', 'browserMajor', 'device.browserSize', 'trafficSource.adwordsClickInfo.adNetworkType', 'geoNetwork.city', 'trafficSource.adwordsClickInfo.page', 'geoNetwork.metro', 'device.mobileDeviceModel', 'trafficSource.referralPath', 'totals.bounces', 'device.language', 'device.browserVersion', 'device.screenColors', 'totals.visits', 'locationZone']
Removed high null columns
Handled new_visits
Handled traffic source
Extracted date features
Dropped unnecessary columns: ['userId', 'sessionId', 'sessionStart', 'trafficSource.campaign', 'trafficSource']
Removed unnecessary column

/tmp/ipykernel_13/742928516.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned = df.replace('not available in demo dataset', np.nan)


Processed training shape: (116023, 362)
Processed test shape: (29006, 362)


In [7]:
X_train, X_val, y_train_, y_val = train_test_split(
    X_train_processed, y_train, test_size=0.05, random_state=42, shuffle=True
)

In [8]:
xgb_model = XGBRegressor(random_state=42, verbosity=0)

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'min_child_weight': [1, 3, 5]
}

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='r2',
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train_)

best_model = grid_search.best_estimator_

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV R2 score: {grid_search.best_score_:.4f}")
print(f"Train R2 score: {best_model.score(X_train, y_train_):.4f}")
print(f"Validation R2 score: {best_model.score(X_val, y_val):.4f}")

# Retrain on full training data with best parameters
best_model.fit(X_train_processed, y_train)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters: {'learning_rate': 0.01, 'max_depth': 9, 'min_child_weight': 3, 'n_estimators': 200}
Best CV R2 score: 0.2967
Train R2 score: 0.6678
Validation R2 score: -0.0033


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [9]:
final_predictions = best_model.predict(X_test_processed)
final_predictions = np.clip(final_predictions, a_min=0, a_max=None) 

In [10]:
# Load the sample submission file
sample_submission = pd.read_csv('/kaggle/input/engage-2-value-from-clicks-to-conversions/sample_submission.csv')
print(f"Sample submission shape: {sample_submission.shape}")
print(f"Sample submission columns: {sample_submission.columns.tolist()}")

# Check if lengths match
print(f"Predictions length: {len(final_predictions)}")
print(f"Sample submission length: {len(sample_submission)}")

# Ensure predictions and sample submission have same length
if len(final_predictions) != len(sample_submission):
    print(f"Warning: Length mismatch - predictions: {len(final_predictions)}, sample: {len(sample_submission)}")
    # Take only the required number of predictions
    predictions_to_use = final_predictions[:len(sample_submission)]
else:
    predictions_to_use = final_predictions

# Create submission dataframe
submission_df = sample_submission.copy()

# Update the purchaseValue column with predictions
submission_df['purchaseValue'] = predictions_to_use

# Check if we need to rename ID column
if 'ID' in submission_df.columns:
    submission_df.rename(columns={'ID': 'id'}, inplace=True)

# Ensure no negative values (clip to 0 minimum)
submission_df['purchaseValue'] = np.clip(submission_df['purchaseValue'], a_min=0, a_max=None)

# Display submission info
print(f"\nSubmission DataFrame Info:")
print(f"Shape: {submission_df.shape}")
print(f"Columns: {submission_df.columns.tolist()}")
print(f"Purchase value stats:")
print(submission_df['purchaseValue'].describe())

# Save the submission file
submission_file = 'submission.csv'
submission_df.to_csv(submission_file, index=False)

print(f"\nSubmission file saved as '{submission_file}'")
print(f"First 10 rows of submission:")
print(submission_df.head(10))

# Verify the file was saved correctly
try:
    test_read = pd.read_csv(submission_file)
    print(f"\nVerification: Successfully read back {len(test_read)} rows from {submission_file}")
except Exception as e:
    print(f"Error reading back submission file: {e}")

Sample submission shape: (29006, 2)
Sample submission columns: ['ID', 'purchaseValue']
Predictions length: 29006
Sample submission length: 29006

Submission DataFrame Info:
Shape: (29006, 2)
Columns: ['id', 'purchaseValue']
Purchase value stats:
count    2.900600e+04
mean     2.541347e+07
std      6.715750e+07
min      0.000000e+00
25%      3.560080e+06
50%      3.572269e+06
75%      1.540791e+07
max      5.057451e+09
Name: purchaseValue, dtype: float64

Submission file saved as 'submission.csv'
First 10 rows of submission:
   id  purchaseValue
0   0   2.795831e+07
1   1   6.575466e+06
2   2   3.561082e+06
3   3   3.660500e+06
4   4   3.720134e+06
5   5   7.321271e+07
6   6   2.206682e+08
7   7   2.613699e+07
8   8   3.560080e+06
9   9   2.329583e+07

Verification: Successfully read back 29006 rows from submission.csv
